# Wet grass / sprinkler / rain example

This example shows how to construct a discrete Bayesian network in Infer.NET and how to learn the parameters of the model (the conditional probability tables) from data.   
The examples from [Imperative Statement Blocks in F#](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html) are included at the end of the notebook.   

### Overview

#### Brief overview of the Wet grass / sprinkler / rain example

Kevin Murphy on [Bayesian networks](https://www.cs.ubc.ca/~murphyk/Bayes/bnintro.html)

If it is cloudy, the probability of rain is higher; and the probability of the sprinkler being used is lower. If either it rained, or the sprinkler was on, the chance that the grass was wet is high (but not certain!); and if both, then even higher.

<img src="https://www.cs.ubc.ca/~murphyk/Bayes/Figures/sprinkler.gif" alt="drawing" width="300"/>

Infer.NET user guide: [Discrete Bayesian network](https://dotnet.github.io/infer/userguide/Discrete%20Bayesian%20network.html )

#### Three ways to infer what you don't know from what you do with Infer.NET.

There are three modes of operation:
* The **parameters** (for the unconditional (cloudy) variable and for the conditional variables (rain, sprinkler, wet grass) conditional probability tables (CBTs)) **are exactly known** - in this case the priors on the parameters are irrelevant as the parameters are directly observed. We can then query the model in a variety of ways.
* The **parameters are unknown**, and we want to infer the parameters from **observations on the primary variables**   
* The **parameters are known with uncertainty** (for example through Bayesian inference), and we want to query the model in the presence of that uncertainty

<img src="https://dotnet.github.io/infer/userguide/WetGrass.png" alt="drawing" width="300"/>

### Source & links

https://dotnet.github.io/infer/userguide/Discrete%20Bayesian%20network.html    
https://github.com/dotnet/infer/blob/master/src/Tutorials/WetGrassSprinklerRain.cs  

http://infernet.azurewebsites.net/default.aspx    
https://www.cs.ubc.ca/~murphyk/Bayes/bnintro.html   
https://social.microsoft.com/Forums/en-US/dcffcf8d-fb15-4236-98fd-9d4a5b19e03a/example-of-bayesian-network-migrated-from-communityresearchmicrosoftcom?forum=infer.net  

## Load & open packages

### Install packages

https://fsprojects.github.io/Paket/

Packages in F# can be installed via `nuget.org` using the Paket dependency manager.
To install packages from NuGet you should first load the Packet package manager:

In [86]:
#load "Paket.fsx"

You can then have Paket install the packages:

In [87]:
Paket.Package
  [   // "Infer.NET" (pre Oct 2018 namespace)
      "Microsoft.ML.Probabilistic"
      "Microsoft.ML.Probabilistic.Compiler"
      //"Microsoft.ML.Probabilistic.FSharp"  // Not on nuget, yet?
      "NETStandard.Library"
  ]

In [88]:
#load "Paket.Generated.Refs.fsx"  // Do we need this?

### `FSharpWrapper`

See  https://github.com/dotnet/infer/blob/master/src/FSharpWrapper/FSharpWrapper.fs   
https://github.com/fsprojects/IfSharp/issues/146 (Thanks for help, Colin Gravill!)   
https://fsprojects.github.io/Paket/github-dependencies.html

In [89]:
Paket.GitHub ["dotnet/infer src/FSharpWrapper/FSharpWrapper.fs"]

In [116]:
//#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

### Assembly search paths and references

In [90]:
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic.Compiler/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/NETStandard.Library/build/netstandard2.0/ref"

In [91]:
#r "Microsoft.ML.Probabilistic"
#r "Microsoft.ML.Probabilistic.Compiler"
#r "netstandard"

### Import declarations for modules or namespaces 

In [92]:
open System

In [93]:
open Microsoft.ML.Probabilistic  
open Microsoft.ML.Probabilistic.Models  
open Microsoft.ML.Probabilistic.Distributions  
open Microsoft.ML.Probabilistic.Factors  // From concept of "factor graph" - Applying functions and operators to variables
open Microsoft.ML.Probabilistic.Math

In [94]:
#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

In [95]:
open Microsoft.ML.Probabilistic.FSharp

## Wet Grass Sprinkler Rain Model

https://github.com/dotnet/infer/blob/master/src/Tutorials/WetGrassSprinklerRain.cs

Note that each of the four main parts of the model require multiple types:
* Dirichlet (prior) **distribution** for the 
* **Random variables** that are the **parameters** (**probabilities** - unconditional or conditional) of the 
* Discrete (actually Bernouilli) **distribution**, that yields
* **Random variables** for the state of the cloud / rain / sprinkler & wet grass

### Methods

Translated from the C#: a couple of helper methods

#### `AddChildFromOneParent`

[Branching on variables to create mixture models](https://dotnet.github.io/infer/userguide/Branching%20on%20variables%20to%20create%20mixture%20models.html)   
[Imperative Statement Blocks in F#](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html)

The random variables are stored in a `VariableArray` to reflect that 
* all of C,R,S,W are conditional to greater or lesser degrees (depending on what is known) and because 
* we like to repeat our experiments many times e.g. N=100 trials

In F# these are typically populated using `Variable.ArrayInit` or    
`v.[r] -> Variable.SomeDistribution(someParameters).ForEach(r)`.   
Even the variable Cloudy, which is unconditional when its parameters (Bernoulli probabilities) are known, is conditional on its driving Dirichlet distribution. 

In [96]:
type RandomVariable = VariableArray<int> // Random variables, e.g. for C (N),S (N*2),R (N*2),W (N*2*2)
type RandomVariableCase = Variable<int> // Could be for single experiment and/or outcome of driving variable

Distribution parameters for (singly) conditional variables: Rain & Sprinker

In [97]:
type ConditionalProbabilityTable = VariableArray<Vector> // Distribution params for S,R

In [98]:
let AddChildFromOneParent 
        (parent:RandomVariable) 
        (cpt:ConditionalProbabilityTable) =
    let r = parent.Range
    let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
                   Variable.SwitchExpr (parent.[i]) (fun (pi:RandomVariableCase) ->  
                       Variable.Discrete(cpt.[pi])))
    child

#### `AddChildFromTwoParents`

Distribution parameters for (doubly conditional) variable WetGrass.   

Class [`VariableArray<TItem, TArray>`](https://dotnet.github.io/infer/apiguide/api/Microsoft.ML.Probabilistic.Models.VariableArray-2.html) is a one-dimensional [jagged variable array](https://dotnet.github.io/infer/userguide/Jagged%20arrays.html).

* The .NET type of a jagged array in Infer.NET is a `VariableArray` with two generic type parameters. The first parameter represents the type of element in the `VariableArray`. In this case we have a `VariableArray` of `VariableArray<Vector>`, so the first type argument is `VariableArray<Vector>`. The second type parameter is the .NET array type of the jagged array - in this case we have an array of array of `Vectors`.

In [99]:
type ConditionalProbabilityTable2D = 
    VariableArray<VariableArray<Vector>, Vector[][]>

In [100]:
let AddChildFromTwoParents 
        (parent1:RandomVariable) 
        (parent2:RandomVariable) 
        (cpt:ConditionalProbabilityTable2D) = 
    let r = parent1.Range
    let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
               Variable.SwitchExpr (parent1.[i]) (fun (pi1:RandomVariableCase) ->  
                   Variable.SwitchExpr (parent2.[i]) (fun (pi2:RandomVariableCase) ->  
                       Variable.Discrete(cpt.[pi1].[pi2]))))
    child       

### Variables

#### `RandomVariables` & `ObservedValues`

Primary random variables:   

In [101]:
type RandomVariables =
    {cloudy:VariableArray<int>;
    sprinkler:VariableArray<int>;
    rain:VariableArray<int>;
    wetGrass:VariableArray<int> }

In [102]:
type ObservedValues =
    {cloudy:array<int>;
    sprinkler:array<int>;
    rain:array<int>;
    wetGrass:array<int> }
type Sample = ObservedValues

#### `DistributionParameters`

Random variables representing the **parameters of the (mostly conditional) distributions** 
of the primary random variables. For child variables, these are 
in the form of **conditional probability tables (CPTs)**   

In [103]:
type DistributionParameters =
    {cloudy:Variable<Vector> ;
    sprinkler:VariableArray<Vector> ;
    rain:VariableArray<Vector> ;
    wetGrass:VariableArray<VariableArray<Vector>, Vector[][]> }

In [104]:
type DistributionParameterValues =  
    {cloudy:Vector;     // probCloudy     2-vector
    sprinkler:Vector[]; // cptSprinkler   2-vector * 2
    rain:Vector[];      // cptRain        2-vector * 2
    wetGrass:Vector[][] // cptWetGrass    2-vector * 2 * 2
    }

#### `PriorDistributions`

Prior distributions for the probability and CPT variables.
The prior distributions are formulated as Infer.NET variables
so that they can be set at runtime without recompiling the model

In [105]:
type PriorDistributions =
    {cloudy:Variable<Dirichlet> ;
    sprinkler:VariableArray<Dirichlet> ;
    rain:VariableArray<Dirichlet> ;
    wetGrass:VariableArray<VariableArray<Dirichlet>, Dirichlet[][]> }

#### `PosteriorDistributions`

Posterior distributions for the probability and CPT variables.   

In [106]:
type PosteriorDistributions =
    {cloudy:Dirichlet ;
    sprinkler:Dirichlet[] ;
    rain:Dirichlet[] ;
    wetGrass:Dirichlet[][] }

#### `Ranges`

In [60]:
type Ranges = {c:Range;s:Range;r:Range;w:Range}

### Assignments

Inference engine

In [107]:
let engine =  InferenceEngine ()

Set up the ranges

In [108]:
let numberOfExamples:Variable<int> = Variable.New<int>().Named("NofE")
let rN:Range = Range(numberOfExamples).Named("N")

Although all the variables in this example have just 2 states (true/false),
the example is formulated in a way that shows how to extend to multiple states

In [109]:
let rC = Range(2).Named("C") // Cloudy
let rS = Range(2).Named("S") // Sprinkler
let rR = Range(2).Named("R") // Rain
let rW = Range(2).Named("W") // Wet

#### Cloudy

Define the priors and the parameters

The values for all variables C,S,R,W have a discrete distribution with probabilities from a Dirichlet (prior) distribution.   
(It's like the outcome comes from the flip of a biased coin, but with the amount of bias being random too - like picking the biased coin from an (infinite) bag of biased coins with the frequencies of coins following a Dirichlet distribution.)

In [110]:
let probCloudyPrior:Variable<Dirichlet> = Variable.New<Dirichlet>().Named("probCloudyPrior")
let probCloudy:Variable<Vector> = Variable<Vector>.Random<Dirichlet>(probCloudyPrior).Named("probCloudy")
probCloudy.SetValueRange(rC)

#### Sprinkler probability table conditioned on cloudiness

[Working with arrays and ranges](https://dotnet.github.io/infer/userguide/Arrays%20and%20ranges.html)

Having defined ranges, you can then **declare a variable array** using the syntax:
```
Variable.Array<T>(Range r1,Range r2,...)
```

In [111]:
let cptSprinklerPrior = Variable.Array<Dirichlet>(rC).Named("cptSprinklerPrior")
let cptSprinkler = Variable.Array<Vector>(rC).Named("cptSprinkler")

At this point, the arrays are declared but not initialized. You can fill an array in two ways: the `SetTo` method (or using an indexer `[]`??)

In [112]:
cptSprinkler.[rC] <- Variable<Vector>.Random<Dirichlet>(cptSprinklerPrior.[rC])
cptSprinkler.SetValueRange(rS)

#### Rain probability table conditioned on cloudiness

In [113]:
let cptRainPrior = Variable.Array<Dirichlet>(rC).Named("cptRainPrior")
let cptRain = Variable.Array<Vector>(rC).Named("cptRain")

In [114]:
cptRain.[rC] <- Variable<Vector>.Random<Dirichlet>(cptRainPrior.[rC])
cptRain.SetValueRange(rR)

#### Wet grass probability table conditioned on sprinkler and rain

In [115]:
let cptWetGrassPrior= 
    Variable.Array<VariableArray<Dirichlet>, Dirichlet[][]>(
        Variable.Array<Dirichlet>(rR), rS).Named("cptWetGrassPrior")
let cptWetGrass = 
    Variable.Array<VariableArray<Vector>, Vector[][]>(
        Variable.Array<Vector>(rR), rS).Named("cptWetGrass")

In [116]:
cptWetGrass.[rS].[rR] <- Variable<Vector>.Random<Dirichlet>(cptWetGrassPrior.[rS].[rR])
cptWetGrass.SetValueRange(rW)

### Now using record types

One way to organize the workflow is to build four-element records to handle the:
* ranges (always 2)
* priors distribution random variables, 
* distribution parameters (conditional probability tables for R,S,W), 
* primary random variables etc. 

In [117]:
let ranges:Ranges = {c=Range(2).Named("C"); s=Range(2).Named("S"); r=Range(2).Named("R"); w=Range(2).Named("R")}

In [72]:
let priors:PriorDistributions = {
    cloudy= Variable.New<Dirichlet>().Named("probCloudyPrior");
    sprinkler= Variable.Array<Dirichlet>(ranges.c).Named("cptSprinklerPrior") ;
    rain= Variable.Array<Dirichlet>(ranges.c).Named("cptRainPrior");
    wetGrass= Variable.Array<VariableArray<Dirichlet>, Dirichlet[][]>(
                    Variable.Array<Dirichlet>(ranges.r), ranges.s).Named("cptWetGrassPrior")}

In [73]:
let parameters:DistributionParameters = {
    cloudy= Variable<Vector>.Random<Dirichlet>(probCloudyPrior).Named("probCloudy");
    sprinkler= Variable.Array<Vector>(ranges.c).Named("cptSprinkler") ;
    rain= Variable.Array<Vector>(ranges.c).Named("cptRain");
    wetGrass= Variable.Array<VariableArray<Vector>, Vector[][]>(
                Variable.Array<Vector>(ranges.r), ranges.s).Named("cptWetGrass")}

In [74]:
parameters.cloudy.SetValueRange(ranges.c)
parameters.sprinkler.[ranges.c] <- Variable<Vector>.Random<Dirichlet>(priors.sprinkler.[ranges.c])
parameters.sprinkler.SetValueRange(ranges.s)
parameters.rain.[ranges.c] <- Variable<Vector>.Random<Dirichlet>(priors.rain.[ranges.c])
parameters.rain.SetValueRange(ranges.r)
parameters.wetGrass.[ranges.s].[ranges.r] <- Variable<Vector>.Random<Dirichlet>(priors.wetGrass.[ranges.s].[ranges.r])
parameters.wetGrass.SetValueRange(ranges.w)

### Functions

#### buildPriors

In [118]:
let buildPriors (rg:Ranges) : PriorDistributions =
    {cloudy= Variable.New<Dirichlet>().Named("probCloudyPrior");
        sprinkler= Variable.Array<Dirichlet>(rg.c).Named("cptSprinklerPrior") ;
        rain= Variable.Array<Dirichlet>(rg.c).Named("cptRainPrior");
        wetGrass= Variable.Array<VariableArray<Dirichlet>, Dirichlet[][]>(
                                Variable.Array<Dirichlet>(rg.r), rg.s).Named("cptWetGrassPrior")}

In [76]:
let priors = buildPriors ranges

#### `buildDistributionParameters`

In [119]:
let buildDistributionParameters (rg:Ranges) (pd:PriorDistributions) : DistributionParameters =
    let probCloudy = Variable<Vector>.Random<Dirichlet>(pd.cloudy).Named("probCloudy")
    let cptSprinkler = Variable.Array<Vector>(rg.c).Named("cptSprinkler") 
    let cptRain = Variable.Array<Vector>(rg.c).Named("cptRain")
    let cptWetGrass = Variable.Array<VariableArray<Vector>, Vector[][]>(
                            Variable.Array<Vector>(rg.r), rg.s).Named("cptWetGrass")
    probCloudy.SetValueRange(rg.c)
    cptSprinkler.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.sprinkler.[rg.c])
    cptSprinkler.SetValueRange(rg.s)
    cptRain.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.rain.[rg.c])
    cptRain.SetValueRange(rg.r)
    cptWetGrass.[rg.s].[rg.r] <- Variable<Vector>.Random<Dirichlet>(pd.wetGrass.[rg.s].[rg.r])
    cptWetGrass.SetValueRange(rg.w)
    {cloudy = probCloudy; sprinkler = cptSprinkler; rain = cptRain; wetGrass = cptWetGrass}        

In [78]:
let parameters = buildDistributionParameters ranges priors

### Define the primary variables

In [120]:
let numberOfExamples:Variable<int> = Variable.New<int>().Named("NofE")
let rN:Range = Range(numberOfExamples).Named("N")

#### Unconditional variable - `Cloudy`

In [121]:
let cloudy = Variable.Array<int>(rN).Named("cloudy")
cloudy.[rN] <- Variable.Discrete(probCloudy).ForEach(rN)

#### Primary (conditional) random variables: `sprinkler`, `rain`, `wetGrass`

In [81]:
let sprinkler:VariableArray<int> = (AddChildFromOneParent cloudy cptSprinkler).Named("sprinkler")
let rain:VariableArray<int> = (AddChildFromOneParent cloudy cptRain).Named("rain")
let wetGrass:VariableArray<int> = (AddChildFromTwoParents sprinkler rain cptWetGrass).Named("wetGrass")

#### Using record types

In [82]:
let rvs:RandomVariables =
    let vC = Variable.Array<int>(rN).Named("cloudy")
    vC.[rN] <- Variable.Discrete(probCloudy).ForEach(rN)
    let vS = (AddChildFromOneParent vC cptSprinkler).Named("sprinkler")
    let vR = (AddChildFromOneParent vC cptRain).Named("rain")
    let vW = (AddChildFromTwoParents vS vR cptWetGrass).Named("wetGrass")
    {cloudy = vC; sprinkler =vS; rain = vR; wetGrass = vW}

#### `buildRandomVariables`

In [83]:
let buildRandomVariables (rN:Range) (dp:DistributionParameters) :RandomVariables =
    let vC = Variable.Array<int>(rN).Named("cloudy")
    vC.[rN] <- Variable.Discrete(dp.cloudy).ForEach(rN)
    let vS = (AddChildFromOneParent vC dp.sprinkler).Named("sprinkler")
    let vR = (AddChildFromOneParent vC dp.rain).Named("rain")
    let vW = (AddChildFromTwoParents vS vR dp.wetGrass).Named("wetGrass")
    {cloudy = vC; sprinkler =vS; rain = vR; wetGrass = vW}

In [84]:
let numberOfExamples = Variable.New<int>().Named("NofE")
let rN = Range(numberOfExamples).Named("N")
let ranges = {c=Range(2).Named("C"); s=Range(2).Named("S"); r=Range(2).Named("R"); w=Range(2).Named("W")}
let priors = buildPriors ranges
let parameters = buildDistributionParameters ranges priors

In [85]:
let rvs = buildRandomVariables rN parameters

### New to F#? New to Infer.NET?

Play with these simple definitions of values

In [160]:
let a:array<int> = [|3|]
let b:int[] =  [|3|]
let c:int[] = a
let d:Vector[][] = [|[|Vector.FromArray(2.,3.)|]|]
let e:array<array<Vector>> = [|[|Vector.FromArray(2.,3.)|]|]
let f:array<array<Vector>> = d

### Workflow

### `WetGrassSprinklerRainModel` - OOP

When using the object oriented programming paradigm in F#, the class is defined using `type`. The constructor is the type definition. The variables (actually immutable values) are made available as properties.

In [161]:
type WetGrassSprinklerRainModel() =
    
    // Private functions (used by constructor)
    let addChildFromOneParent 
            (parent:RandomVariable) 
            (cpt:ConditionalProbabilityTable) =
        let r = parent.Range
        let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
                       Variable.SwitchExpr (parent.[i]) (fun (pi:RandomVariableCase) ->  
                           Variable.Discrete(cpt.[pi])))
        child
        
    let addChildFromTwoParents 
            (parent1:RandomVariable) 
            (parent2:RandomVariable) 
            (cpt:ConditionalProbabilityTable2D) = 
        let r = parent1.Range
        let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
                   Variable.SwitchExpr (parent1.[i]) (fun (pi1:RandomVariableCase) ->  
                       Variable.SwitchExpr (parent2.[i]) (fun (pi2:RandomVariableCase) ->  
                           Variable.Discrete(cpt.[pi1].[pi2]))))
        child    
        
    let buildPriors (rg:Ranges) : PriorDistributions =
        {cloudy= Variable.New<Dirichlet>().Named("probCloudyPrior");
            sprinkler= Variable.Array<Dirichlet>(rg.c).Named("cptSprinklerPrior") ;
            rain= Variable.Array<Dirichlet>(rg.c).Named("cptRainPrior");
            wetGrass= Variable.Array<VariableArray<Dirichlet>, Dirichlet[][]>(
                                    Variable.Array<Dirichlet>(rg.r), rg.s).Named("cptWetGrassPrior")}

    let buildDistributionParameters (rg:Ranges) (pd:PriorDistributions) : DistributionParameters =
        let probCloudy = Variable<Vector>.Random<Dirichlet>(pd.cloudy).Named("probCloudy")
        let cptSprinkler = Variable.Array<Vector>(rg.c).Named("cptSprinkler") 
        let cptRain = Variable.Array<Vector>(rg.c).Named("cptRain")
        let cptWetGrass = Variable.Array<VariableArray<Vector>, Vector[][]>(
                                Variable.Array<Vector>(rg.r), rg.s).Named("cptWetGrass")
        do probCloudy.SetValueRange(rg.c)
        do cptSprinkler.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.sprinkler.[rg.c])
        do cptSprinkler.SetValueRange(rg.s)
        do cptRain.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.rain.[rg.c])
        do cptRain.SetValueRange(rg.r)
        do cptWetGrass.[rg.s].[rg.r] <- Variable<Vector>.Random<Dirichlet>(pd.wetGrass.[rg.s].[rg.r])
        do cptWetGrass.SetValueRange(rg.w)
        {cloudy = probCloudy; sprinkler = cptSprinkler; rain = cptRain; wetGrass = cptWetGrass}        

    let buildRandomVariables (rN:Range) (dp:DistributionParameters) :RandomVariables =
        let vC = Variable.Array<int>(rN).Named("cloudy")
        do vC.[rN] <- Variable.Discrete(dp.cloudy).ForEach(rN)
        let vS = (addChildFromOneParent vC dp.sprinkler).Named("sprinkler")
        let vR = (addChildFromOneParent vC dp.rain).Named("rain")
        let vW = (addChildFromTwoParents vS vR dp.wetGrass).Named("wetGrass")
        {cloudy = vC; sprinkler =vS; rain = vR; wetGrass = vW}
     
    // Private fields
    let ranges:Ranges = {c=Range(2).Named("C"); s=Range(2).Named("S"); r=Range(2).Named("R"); w=Range(2).Named("R")}
    let numberOfExamples:Variable<int> = Variable.New<int>().Named("NofE")
    let rN:Range = Range(numberOfExamples).Named("N")
    let priors = buildPriors ranges
    let parameters = buildDistributionParameters ranges priors
    let primaryRandomVariables = buildRandomVariables rN parameters
    let engine = InferenceEngine ()

    
    // Properties
    member this.Ranges = ranges
    member this.NumberOfExamples = numberOfExamples
    member this.RangeN = rN
    member this.Priors = priors
    member this.Parameters = parameters
    member this.PrimaryRandomVariables = primaryRandomVariables
    member this.Engine = engine

In [162]:
let model = WetGrassSprinklerRainModel ()  

### `WetGrassSprinklerRainModel` - functional

When using a more functional paradigm for organizing the code, the functions take greater precedence than the data, & are organised into a module. By using `open ModuleName` the functions become available without requiring a fully qualified `ModuleName.functionName`.

In [163]:
module WetGrassSprinklerRainModel2 =
    // Functions
    let addChildFromOneParent 
            (parent:RandomVariable) 
            (cpt:ConditionalProbabilityTable) =
        let r = parent.Range
        let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
                       Variable.SwitchExpr (parent.[i]) (fun (pi:RandomVariableCase) ->  
                           Variable.Discrete(cpt.[pi])))
        child
        
    let addChildFromTwoParents 
            (parent1:RandomVariable) 
            (parent2:RandomVariable) 
            (cpt:ConditionalProbabilityTable2D) = 
        let r = parent1.Range
        let (child:RandomVariable) = Variable.ArrayInit r (fun (i:Range)->  
                   Variable.SwitchExpr (parent1.[i]) (fun (pi1:RandomVariableCase) ->  
                       Variable.SwitchExpr (parent2.[i]) (fun (pi2:RandomVariableCase) ->  
                           Variable.Discrete(cpt.[pi1].[pi2]))))
        child    
    
    let buildPriors (rg:Ranges) : PriorDistributions =
        {cloudy= Variable.New<Dirichlet>().Named("probCloudyPrior");
            sprinkler= Variable.Array<Dirichlet>(rg.c).Named("cptSprinklerPrior") ;
            rain= Variable.Array<Dirichlet>(rg.c).Named("cptRainPrior");
            wetGrass= Variable.Array<VariableArray<Dirichlet>, Dirichlet[][]>(
                                    Variable.Array<Dirichlet>(rg.r), rg.s).Named("cptWetGrassPrior")}

    let buildDistributionParameters (rg:Ranges) (pd:PriorDistributions) : DistributionParameters =
        let probCloudy = Variable<Vector>.Random<Dirichlet>(pd.cloudy).Named("probCloudy")
        let cptSprinkler = Variable.Array<Vector>(rg.c).Named("cptSprinkler") 
        let cptRain = Variable.Array<Vector>(rg.c).Named("cptRain")
        let cptWetGrass = Variable.Array<VariableArray<Vector>, Vector[][]>(
                                Variable.Array<Vector>(rg.r), rg.s).Named("cptWetGrass")
        probCloudy.SetValueRange(rg.c)
        cptSprinkler.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.sprinkler.[rg.c])
        cptSprinkler.SetValueRange(rg.s)
        cptRain.[rg.c] <- Variable<Vector>.Random<Dirichlet>(pd.rain.[rg.c])
        cptRain.SetValueRange(rg.r)
        cptWetGrass.[rg.s].[rg.r] <- Variable<Vector>.Random<Dirichlet>(pd.wetGrass.[rg.s].[rg.r])
        cptWetGrass.SetValueRange(rg.w)
        {cloudy = probCloudy; sprinkler = cptSprinkler; rain = cptRain; wetGrass = cptWetGrass}        

    let buildRandomVariables (rN:Range) (dp:DistributionParameters) :RandomVariables =
        let vC = Variable.Array<int>(rN).Named("cloudy")
        vC.[rN] <- Variable.Discrete(dp.cloudy).ForEach(rN)
        let vS = (addChildFromOneParent vC dp.sprinkler).Named("sprinkler")
        let vR = (addChildFromOneParent vC dp.rain).Named("rain")
        let vW = (addChildFromTwoParents vS vR dp.wetGrass).Named("wetGrass")
        {cloudy = vC; sprinkler =vS; rain = vR; wetGrass = vW}

In [164]:
open WetGrassSprinklerRainModel2
let ranges:Ranges = {c=Range(2).Named("C"); s=Range(2).Named("S"); r=Range(2).Named("R"); w=Range(2).Named("R")}
let numberOfExamples:Variable<int> = Variable.New<int>().Named("NofE")
let rN:Range = Range(numberOfExamples).Named("N")
let priors = buildPriors ranges
let parameters = buildDistributionParameters ranges priors
let rvs = buildRandomVariables rN parameters
let engine =  InferenceEngine ()

### More methods: `learnParameters` etc.

#### `setDistributionParameterValues`

In [122]:
let setDistributionParameterValues (): DistributionParameterValues =
    let probCloudy:Vector = Vector.FromArray(0.5, 0.5)
    let cptSprinkler:Vector[] = 
        [| [|0.1; 0.9|] (* cloudy *); [|0.5; 0.5|] (* not cloudy *) |] 
        |> Array.map (fun a -> Vector.FromArray a)
    let cptRain:Vector[] =      
        [| [|0.8; 0.2|] (* cloudy *); [|0.2; 0.8|] (* not cloudy *) |] 
        |> Array.map (fun a -> Vector.FromArray a)
    let cptWetGrass:Vector[][] = 
        [|  [|  [| 0.99; 0.01|] (* rain *); 
                [| 0.9 ; 0.1 |] (* not rain *) |];     // Sprinkler
            [|  [| 0.9 ; 0.1 |] (* rain *); 
                [| 0.0 ; 1.0 |] (* not rain *) |] |]   // Not sprinkler  
        |> Array.map (fun a -> a |> Array.map Vector.FromArray)
    {cloudy=probCloudy; sprinkler=cptSprinkler; rain=cptRain; wetGrass=cptWetGrass}

In [166]:
let dpv = setDistributionParameterValues ()

In [167]:
dpv

{cloudy = seq [0.5; 0.5];
 sprinkler = [|seq [0.1; 0.9]; seq [0.5; 0.5]|];
 rain = [|seq [0.8; 0.2]; seq [0.2; 0.8]|];
 wetGrass =
  [|[|seq [0.99; 0.01]; seq [0.9; 0.1]|]; [|seq [0.9; 0.1]; seq [0.0; 1.0]|]|];}

In [123]:
let sample  (numData:int)
            (dpv:DistributionParameterValues) : Sample =   
    let z = Array2D.init numData 4 (fun _ j -> 
        let cloudy =  Discrete.Sample(dpv.cloudy)
        let sprinkler = Discrete.Sample(dpv.sprinkler.[cloudy])
        let rain = Discrete.Sample(dpv.rain.[cloudy])
        let wetGrass = Discrete.Sample(dpv.wetGrass.[sprinkler].[rain])
        [|cloudy; sprinkler; rain; wetGrass|].[j])
    {cloudy = z.[*,0]; sprinkler = z.[*,1]; rain=z.[*,2]; wetGrass = z.[*,3]}

In [195]:
let sample100 = sample 100 dpv

#### `learnParameters`

Learns the parameters of the cloud/sprinkler/rain example
* Given observed values for the RVs & observed values for the priors
* Given observed values for the RVs assuming uniform distributions for the priors

In [58]:
let learnParameters (engine:InferenceEngine)
                    (numberOfExamples:Variable<int>)
                    (rvs:RandomVariables)
                    (pri:PriorDistributions)
                    (obs:ObservedValues)
                    (pos:PosteriorDistributions):PosteriorDistributions =
    do numberOfExamples.ObservedValue <- obs.cloudy.Length
    do rvs.cloudy.ObservedValue <- obs.cloudy
    do rvs.sprinkler.ObservedValue <- obs.sprinkler
    do rvs.rain.ObservedValue <- obs.rain
    do rvs.wetGrass.ObservedValue <- obs.wetGrass
    
    do pri.cloudy.ObservedValue <- pos.cloudy
    do pri.sprinkler.ObservedValue <- pos.sprinkler
    do pri.rain.ObservedValue <- pos.rain
    do pri.wetGrass.ObservedValue <- pos.wetGrass

    // Inference 
    {cloudy=engine.Infer<Dirichlet>(pri.cloudy);  
        sprinkler=engine.Infer<Dirichlet[]>(pri.sprinkler);  
        rain=engine.Infer<Dirichlet[]>(pri.rain);  
        wetGrass=engine.Infer<Dirichlet[][]>(pri.wetGrass)}

In [198]:
let uniformPriors:PosteriorDistributions = 
    let probCloudyPrior = Dirichlet.Uniform(2)
    let dirUnifArray = Array.init 2 (fun _ -> Dirichlet.Uniform(2)) 
    let dirUnifArrayArray = Array.init 2 (fun _ -> dirUnifArray) 
    {cloudy=probCloudyPrior; sprinkler=dirUnifArray; rain=dirUnifArray; wetGrass=dirUnifArrayArray}

In [199]:
learnParameters engine numberOfExamples rvs priors sample100 uniformPriors

This expression was expected to have type
    'FSI_0019.RandomVariables'    
but here has type
    'FSI_0149.RandomVariables'    
This expression was expected to have type
    'FSI_0023.PriorDistributions'    
but here has type
    'FSI_0153.PriorDistributions'    
This expression was expected to have type
    'ObservedValues'    
but here has type
    'Sample'    
This expression was expected to have type
    'FSI_0024.PosteriorDistributions'    
but here has type
    'FSI_0235.PosteriorDistributions'    

#### `probRain`

Returns the probability of Rain given optional readings on
cloudiness, sprinkler, and wetness of grass, and given prior distributions
over the parameters. Priors may be manually set, or may be the posteriors
from learning the parameters.

https://github.com/dotnet/infer/blob/master/src/Tutorials/WetGrassSprinklerRain.cs

In [59]:
let probRain (cloudy:Option<int>) (sprinkler:Option<int>) (wet:Option<int>) = 
    match cloudy with 
    | Some cloudy -> printfn "the valid value is %A" cloudy
    | None -> printfn "the value is None" 
    ()

In [60]:
probRain (Some 3) None

the valid value is 3


In [61]:
NumberOfExamples.ObservedValue = 1;

The value, namespace, type or module 'NumberOfExamples' is not defined. Maybe you want one of the following:
   numberOfExamples

In [62]:
(*
if (cloudy.HasValue)
            {
                Cloudy.ObservedValue = new int[] { cloudy.Value };
            }
            else
            {
                Cloudy.ClearObservedValue();
            }

            if (sprinkler.HasValue)
            {
                Sprinkler.ObservedValue = new int[] { sprinkler.Value };
            }
            else
            {
                Sprinkler.ClearObservedValue();
            }

            if (wet.HasValue)
            {
                WetGrass.ObservedValue = new int[] { wet.Value };
            }
            else
            {
                WetGrass.ClearObservedValue();
            }

            Rain.ClearObservedValue();

            ProbCloudyPrior.ObservedValue = probCloudyPrior;
            CPTSprinklerPrior.ObservedValue = cptSprinklerPrior;
            CPTRainPrior.ObservedValue = cptRainPrior;
            CPTWetGrassPrior.ObservedValue = cptWetGrassPrior;

            // Inference
            var rainPosterior = Engine.Infer<Discrete[]>(Rain);

            // In this example, index 0 is true and index 1 is false
            return rainPosterior[0].GetProbs()[0];
        } *)

## Testing the model

### Usage 1: Query the model when we know the parameters exactly

#### `setDistributionParameterValues`

In [64]:
let setDistributionParameterValues (): DistributionParameterValues =
    let probCloudy:Vector = Vector.FromArray(0.5, 0.5)
    let cptSprinkler:Vector[] = [| [|0.1; 0.9|] (* cloudy *); [|0.5; 0.5|] (* not cloudy *) |] |> Array.map (fun a -> Vector.FromArray a)
    let cptRain:Vector[] =      [| [|0.8; 0.2|] (* cloudy *); [|0.2; 0.8|] (* not cloudy *) |] |> Array.map (fun a -> Vector.FromArray a)
    let cptWetGrass:Vector[][] = 
        [|  [|  [| 0.99; 0.01|] (* rain *); 
                [| 0.9 ; 0.1 |] (* not rain *) |];     // Sprinkler
            [|  [| 0.9 ; 0.1 |] (* rain *); 
                [| 0.0 ; 1.0 |] (* not rain *) |] |]   // Not sprinkler  
        |> Array.map (fun a -> a |> Array.map Vector.FromArray)
    {cloudy=probCloudy; sprinkler=cptSprinkler; rain=cptRain; wetGrass=cptWetGrass}

In [94]:
let dpv = setDistributionParameterValues ()

In [66]:
let probRainGivenWetGrass = model.ProbRain(null, null, 0, probCloudy, cptSprinkler, cptRain, cptWetGrass);
let probRainGivenWetGrassNotCloudy = model.ProbRain(1, null, 0, probCloudy, cptSprinkler, cptRain, cptWetGrass);

printf "P(rain | grass is wet)              = %A" probRainGivenWetGrass
printf "P(rain | grass is wet, not cloudy ) = %A" probRainGivenWetGrassNotCloudy

The field, constructor or member 'ProbRain' is not defined.
The field, constructor or member 'ProbRain' is not defined.

### Usage 2: Learn posterior distributions for the parameters

Learning parameters from data (uniform prior)

In [95]:
let sample10 = sample 10 dpv

In [98]:
sample10.[0]

[|1; 1; 1; 1|]

See if we can recover the parameters from the data - assume uniform priors

In [68]:
model.LearnParameters(sample[0], sample[1], sample[2], sample[3])

The field, constructor or member 'LearnParameters' is not defined.

The posteriors are distributions over the probabilities and CPTs. Print out the means of these
distributions, and compare with the ground truth

In [69]:
printf "Prob. Cloudy:                              Ground truth: %A, Inferred: %A", 0.5, model.ProbCloudyPosterior.GetMean()[0]
printf "Prob. Sprinkler | Cloudy:                  Ground truth: %A, Inferred: %A", 0.1, model.CPTSprinklerPosterior[0].GetMean()[0]
printf "Prob. Sprinkler | Not Cloudy:              Ground truth: %A, Inferred: %A", 0.5, model.CPTSprinklerPosterior[1].GetMean()[0]
printf "Prob. Rain      | Cloudy:                  Ground truth: %A, Inferred: %A", 0.8, model.CPTRainPosterior[0].GetMean()[0]
printf "Prob. Rain      | Not Cloudy:              Ground truth: %A, Inferred: %A", 0.2, model.CPTRainPosterior[1].GetMean()[0]
printf "Prob. Wet Grass | Sprinkler, Rain:         Ground truth: %A, Inferred: %A", 0.99, model.CPTWetGrassPosterior[0][0].GetMean()[0]
printf "Prob. Wet Grass | Sprinkler, Not Rain      Ground truth: %A, Inferred: %A", 0.9, model.CPTWetGrassPosterior[0][1].GetMean()[0]
printf "Prob. Wet Grass | Not Sprinkler, Rain:     Ground truth: %A, Inferred: %A", 0.9, model.CPTWetGrassPosterior[1][0].GetMean()[0]
printf "Prob. Wet Grass | Not Sprinkler, Not Rain: Ground truth: %A, Inferred: %A", 0.0, model.CPTWetGrassPosterior[1][1].GetMean()[0]

Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
The field, constructor or member 'ProbCloudyPosterior' is not defined.
The field, constructor or member 'CPTSprinklerPosterior' is not defined.
The field, constructor or member 'CPTSprinklerPosterior' is not defined.
The field, constructor or member 'CPTRainPosterior' is not defined.
The field, constructor or member 'CPTRainPosterior' is not defined.
The field, constructor or member 'CPTWetGrassPosterior' is not defined.
The fie

### Usage 3: Querying the model taking into account uncertainty of the parameters

Querying the model with uncertain ground truth

In [70]:
// Use posteriors we have just learnt
let probRainGivenWetGrass1 = model.ProbRain(null, null, 0, model.ProbCloudyPosterior, model.CPTSprinklerPosterior, model.CPTRainPosterior, model.CPTWetGrassPosterior)
let probRainGivenWetGrassNotCloudy1 = model.ProbRain(1, null, 0, model.ProbCloudyPosterior, model.CPTSprinklerPosterior, model.CPTRainPosterior, model.CPTWetGrassPosterior)

The field, constructor or member 'ProbRain' is not defined.
The field, constructor or member 'ProbRain' is not defined.

P(rain | grass is wet) 

In [71]:
probRainGivenWetGrass1

The value or constructor 'probRainGivenWetGrass1' is not defined. Maybe you want one of the following:
   probRain

P(rain | grass is wet, not cloudy )

In [72]:
probRainGivenWetGrassNotCloudy1

The value or constructor 'probRainGivenWetGrassNotCloudy1' is not defined.

## Infer.NET user guide

### [Jagged arrays](https://dotnet.github.io/infer/userguide/Jagged%20arrays.html)

Source: [Jagged arrays](https://dotnet.github.io/infer/userguide/Jagged%20arrays.html)

In many models data will not be in the form of a flat single dimensional or a multi-dimension array. For example, items in a data array may have a variable number of features. In this case the observed data is represented as an array of arrays of values; in C# this is referred to as a jagged array, and Infer.NET has an API which allows creation of jagged variable arrays.

The simplest jagged array is a 1D array of 1D arrays. The 
* outer array can be indexed by a standard range, but the 
* inner array is variable in size, and the size itself is a function of the outer index. 

Here is an example the illustrates the syntax:

In [73]:
let sizes:array<int> = [| 2; 3 |]  
let item:Range = Range(sizes.Length).Named("item")  // Standard range of 2 items is outer loop
let sizesVar:VariableArray<int> = Variable.Constant(sizes, item).Named("sizes")  
let feature:Range =  Range(sizesVar.[item]).Named("feature")  // Inner range "feature" is variable in size
let x:VariableArray<VariableArray<float>,float[][]>
        = Variable.Array<VariableArray<float>,float[][]>(Variable.Array<float>(feature), item).Named("x");

In this case, we specify the `sizes` as a constant variable array as shown in the first three lines. The inner range can then be defined using the natural expression for the size of the inner range. The jagged array is then created on the final line using the standard `Array` constructor where the first argument is the array element (created as variable array of doubles ranging over the inner range), and the second argument is the outer range. 

The .NET type of a jagged array in Infer.NET is a `VariableArray` with two generic type parameters. The first parameter represents the type of element in the `VariableArray`. In this case we have a `VariableArray` of `VariableArray<double>`, so the first type argument is `VariableArray<double>`. The second type parameter is the .NET array type of the jagged array - in this case we have an array of array of doubles.

The size of `x` is $2+3$. We say `x` has two items, of which, one has 2 features & one has 3 features?!

We can now use the ranges to index and loop over our jagged array in a natural manner:

In [74]:
let xPrior:Gaussian = Gaussian(1.2, 3.4) 
x.[item].[feature] <- Variable.Random(xPrior).ForEach(item, feature)  // Assign all 5 item-features (!)
Variable.ConstrainPositive(x.[item].[feature])   // Constrain all 5 item-features (!)

## Infer.NET user guide : FSharp Wrapper

### Imperative Statement Blocks in F#

[Imperative Statement Blocks in F#](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html)

The module Variable contains various functions to replace the imperative statement blocks in Infer.NET, these include `ForeachBlock`, `SwitchBlock` and `IfBlock` methods.

### `ForEach` blocks

The `Variable.ForeachBlock` method which returns a `ForEachBlock` used for looping over a Range of values in a similar manner to a `foreach` loop in C#. This is
a function with the following type signature: `Variable.ForEachBlock: RRange->(Range->unit)->unit`

In [75]:
//using the Foreach Block  
let pixel = Range(10)  
let bools = Variable.Array<bool>(pixel)

Variable.ForeachBlock pixel ( fun pixel -> bools.[pixel] <- Variable.Bernoulli(0.7) ||| Variable.Bernoulli(0.4))

It should be noted that the `Variable.ArrayInit` method or the `Variable.AssignVariableArray` method can also be used to assign the values of a `VariableArray` as shown in the section [Variable Arrays](https://dotnet.github.io/infer/userguide/Variable%20Arrays%20in%20FSharp.html)

### Switch blocks

#### `Variable.SwitchBlock` - for single variables

The `Variable.SwitchBlock` method which returns a `SwitchBlock` used for **branching on the values of a variable**. This is a function with the following type signature: `Variable.SwitchBlock: Variable<int>->(Variable<int>->unit)->unit`

Model a (scalar) 2 component Gaussian mixture

In [76]:
//using the Switch Block - to model a 2 component (scalar) Gaussian mixture
let mixtureSize = 2  
let k2 = Range(mixtureSize)  
let c:Variable<int> = Variable.Discrete(k2, [|0.5;0.5|])  // c is a discrete random variable with values {0,1}
let means2:VariableArray<float> = Variable.Observed( [|1.0;2.0|], k2)  // Constant means for each Gaussian
let x:Variable<float> = Variable.New<double>()  // x is a univariate Gaussian

In [77]:
Variable.SwitchBlock c (fun _ -> 
    let _ = x.SetTo(Variable.GaussianFromMeanAndVariance(means2.[c], 1.0))  
    ()  
)

#### `Variable.SwitchExpr` - for (arrays of) multiple variables (e.g. for repeated samples)

There is also another Switch construct which returns a `Variable<'a>` and can be used to initialise or set the values of `VariableArray`s. The function `Variable.SwitchExpr` has the following type signature: `VVariable<int> -> (Variable<int>->Variable<'a>)->Variable<int>`

Model 100 outcomes of a 2-component mixture Wishart distributed variable

In [78]:
//using the SwitchExpr Block - to model 100 outcomes of a 2-component mixture Wishart distributed variable
let k =  Range(2).Named("k")

In [79]:
// Loop over mixture components - parameters are constant, so don't vary based on prior distribution, yet
let means:VariableArray<Vector> = 
    Variable.ArrayInit k (fun k ->  
        Variable.VectorGaussianFromMeanAndPrecision( Vector.Constant(2,0.0), PositiveDefiniteMatrix.Identity(2)))  

In [80]:
let precs:VariableArray<PositiveDefiniteMatrix> =  
    Variable.ArrayInit k (fun k -> 
        Variable.WishartFromShapeAndScale(1.0,  
            PositiveDefiniteMatrix.Identity(2)))  

Random weights for two mixture components are [Dirichlet ](https://en.wikipedia.org/wiki/Dirichlet_distribution). Dirichlet continuous multivariate probability distributions are parameterized by a vector of positive reals), and have [applications](https://en.wikipedia.org/wiki/Dirichlet_distribution#Applications) as the prior distribution of categorical variables or multinomial variables in Bayesian mixture models and other hierarchical Bayesian models.

In [81]:
let weights = Variable.Dirichlet(k,[|1.0; 1.0|])

Range `n` for the outcomes to store in the variable array.

In [82]:
let n = Range(100).Named("n")

Define `z` to be array of (random) variables for the $n=100$ integer $\{0,1\}$ values from the discrete distribution, with weights from the Dirichlet distribution.

In [83]:
let z:VariableArray<int> = Variable.ArrayInit n (fun i -> Variable.Discrete(weights))  

Blocks are:
* Outer block is loop over the `Range` of $n=100$ outcomes   
* Inner block explores the cases of the switch statement, selecting only the mean & precision for the value of the (discrete, integer-valued random) variable $z_i$

In [84]:
let data = Variable.ArrayInit n (fun (i:Range)->  
               Variable.SwitchExpr (z.[i]) (fun (zi:Variable<int>) ->  
                   Variable.VectorGaussianFromMeanAndPrecision(means.[zi], precs.[zi])))

### If Blocks

The `Variable.IfBlock` method returns an `IfBlock` for creating if then else statements which are dependant on the value of a `Variable<bool>`. This is a function with type signature: `Variable.IfBlock: Variable<bool>->(Variable<bool>->unit)->Variable<bool>->unit)->unit` where, the first argument is a boolean variable, the second argument is a function to be applied if the Boolean Variable has value true and the third argument is a function to be applied if the Boolean Variable has value false. Here the If block is used in the Clinical trial tutorial shown below

In [85]:
// Data from a clinical trial  
let controlGroup = Variable.Observed<bool>([|false; false; true; false; false|])  
let treatedGroup = Variable.Observed<bool>([|true; false; true; true; true |])  
let i = controlGroup.Range  
let j = treatedGroup.Range  
// Prior on being an effective treatment  
let isEffective = Variable.Bernoulli(0.5).Named("isEffective");  
let probIfTreated = ref (Variable.New<float>())  
let probIfControl = ref (Variable.New<float>())

In [86]:
// If Block function  
let f1 (vb1: Variable<bool>) =  
    probIfControl := Variable.Beta(1.0, 1.0).Named("probIfControl")  
    let controlGroup = Variable.AssignVariableArray controlGroup i (fun i ->
        Variable.Bernoulli(!probIfControl)) 
    let treatedGroup = Variable.AssignVariableArray treatedGroup j (fun j ->
        Variable.Bernoulli(!probIfTreated))  
    ()  

In [87]:
// IfNot Block function  
let f2 (vb2: Variable<bool>) =  
    let probAll = Variable.Beta(1.0, 1.0).Named("probAll")  
    let controlGroup = Variable.AssignVariableArray controlGroup i (fun i ->Variable.Bernoulli(probAll))  
    let treatedGroup = Variable.AssignVariableArray treatedGroup j (fun j ->Variable.Bernoulli(probAll))  
    ()  

In [88]:
// Call IfBlock  
let _ = Variable.IfBlock isEffective f1 f2